# 服务器使用和工作流

本教程介绍如何在服务器上运行机器学习代码，包括私有集群和使用SLURM的公共集群。

## 内容概览
1. 服务器连接和基本操作
2. 文件传输（上传/下载）
3. 环境配置和包管理
4. 任务提交和监控
5. SLURM作业调度系统
6. 最佳实践和常见问题

## 1. 服务器连接

### SSH连接
```bash
# 基本连接
ssh username@server_address

# 指定端口
ssh -p 2222 username@server_address

# 使用密钥连接
ssh -i ~/.ssh/private_key username@server_address

# 保持连接活跃
ssh -o ServerAliveInterval=60 username@server_address
```

### 配置SSH免密登录
```bash
# 1. 生成SSH密钥对
ssh-keygen -t rsa -b 4096 -C "your_email@example.com"

# 2. 复制公钥到服务器
ssh-copy-id username@server_address

# 3. 或手动添加
cat ~/.ssh/id_rsa.pub | ssh username@server_address "mkdir -p ~/.ssh && cat >> ~/.ssh/authorized_keys"
```

## 2. 文件传输

### SCP命令
```bash
# 上传文件
scp local_file.py username@server:/remote/path/

# 上传文件夹
scp -r local_folder/ username@server:/remote/path/

# 下载文件
scp username@server:/remote/file.py ./local_path/

# 下载文件夹
scp -r username@server:/remote/folder/ ./local_path/
```

### Rsync同步
```bash
# 增量同步（推荐）
rsync -avz --progress local_folder/ username@server:/remote/path/

# 排除特定文件
rsync -avz --exclude='*.log' --exclude='__pycache__' local_folder/ username@server:/remote/path/
```

In [ ]:
# 服务器环境检查脚本
import subprocess
import sys
import os
import psutil

def check_server_environment():
    """检查服务器环境信息"""
    print("=== 服务器环境信息 ===")
    
    # 系统信息
    print(f"操作系统: {os.uname().sysname} {os.uname().release}")
    print(f"主机名: {os.uname().nodename}")
    print(f"Python版本: {sys.version}")
    
    # 硬件信息
    print(f"CPU核心数: {psutil.cpu_count()}")
    print(f"内存总量: {psutil.virtual_memory().total / (1024**3):.1f} GB")
    print(f"可用内存: {psutil.virtual_memory().available / (1024**3):.1f} GB")
    
    # GPU信息
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.used', '--format=csv,noheader,nounits'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            print("GPU信息:")
            for line in result.stdout.strip().split('\n'):
                print(f"  {line}")
        else:
            print("未检测到GPU或nvidia-smi不可用")
    except FileNotFoundError:
        print("nvidia-smi命令未找到")
    
    # 磁盘使用情况
    disk_usage = psutil.disk_usage('/')
    print(f"磁盘使用: {disk_usage.used / (1024**3):.1f} GB / {disk_usage.total / (1024**3):.1f} GB")
    
    # 环境变量
    important_vars = ['CUDA_VISIBLE_DEVICES', 'PATH', 'PYTHONPATH']
    print("\n重要环境变量:")
    for var in important_vars:
        value = os.environ.get(var, '未设置')
        print(f"  {var}: {value[:100]}{'...' if len(str(value)) > 100 else ''}")

# 运行检查
check_server_environment()